In [16]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import dates
import datetime

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots


In [17]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [18]:
# dictionary = {'*':''}
# train = train.replace(dictionary, regex=True, inplace=True)
train = train[train['Country/Region'] == "US"]
test = test[test['Country/Region'] == "US"]
# train = train[train['Country/Region'] == 'Taiwan*']
# test = test[test['Country/Region'] == 'Taiwan*']
# np.array(range(len(test))).reshape((-1,1))
train.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
12198,19903,Alabama,US,32.3182,-86.9023,2020-01-22,0.0,0.0
12199,19904,Alabama,US,32.3182,-86.9023,2020-01-23,0.0,0.0
12200,19905,Alabama,US,32.3182,-86.9023,2020-01-24,0.0,0.0
12201,19906,Alabama,US,32.3182,-86.9023,2020-01-25,0.0,0.0
12202,19907,Alabama,US,32.3182,-86.9023,2020-01-26,0.0,0.0


In [19]:

for country in train['Country/Region'].unique():
    print ('training model for country ==>'+country)
    country_pd_train = train[train['Country/Region']==country]
    country_pd_test = test[test['Country/Region']==country]
    
    print(country_pd_train['Province/State'].isna().unique()) 
    
    
    if country_pd_train['Province/State'].isna().unique()==True:   
        x = np.array(range(len(country_pd_train))).reshape((-1,1))
        y = country_pd_train['ConfirmedCases']
        model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                         ('linear', LinearRegression(fit_intercept=False))])
        model = model.fit(x, y)
#         print(len(country_pd_test))
#         print(range(len(country_pd_test)))
#         print(np.array(range(len(country_pd_test)))+50)
        
        
        
        predict_x = (np.array(range(len(country_pd_test)))+50).reshape((-1,1))
        
        test.loc[test['Country/Region']==country,'ConfirmedCases'] = model.predict(predict_x)
    
    else:
        for state in country_pd_train['Province/State'].unique():
            state_pd = country_pd_train[country_pd_train['Province/State']==state] 
            state_pd_test = country_pd_test[country_pd_test['Province/State']==state] 
            x = np.array(range(len(state_pd))).reshape((-1,1))
            y = state_pd['ConfirmedCases']
            model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                         ('linear', LinearRegression(fit_intercept=False))])
            model = model.fit(x, y)
            predict_x = (np.array(range(len(state_pd_test)))+50).reshape((-1,1))
            test.loc[(test['Country/Region']==country)&(test['Province/State']==state),'ConfirmedCases'] = model.predict(predict_x)

training model for country ==>US
[False]


In [20]:
for country in train['Country/Region'].unique():
    print ('training model for country ==>'+country)
    country_pd_train = train[train['Country/Region']==country]
    country_pd_test = test[test['Country/Region']==country]
    if country_pd_train['Province/State'].isna().unique()==True:
        x = np.array(range(len(country_pd_train))).reshape((-1,1))
        y = country_pd_train['Fatalities']
        model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                         ('linear', LinearRegression(fit_intercept=False))])
        model = model.fit(x, y)
        predict_x = (np.array(range(len(country_pd_test)))+50).reshape((-1,1))
        test.loc[test['Country/Region']==country,'Fatalities'] = model.predict(predict_x)
    else:
        for state in country_pd_train['Province/State'].unique():
            state_pd = country_pd_train[country_pd_train['Province/State']==state] 
            state_pd_test = country_pd_test[country_pd_test['Province/State']==state] 
            x = np.array(range(len(state_pd))).reshape((-1,1))
            y = state_pd['Fatalities']
            model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                         ('linear', LinearRegression(fit_intercept=False))])
            model = model.fit(x, y)
            predict_x = (np.array(range(len(state_pd_test)))+50).reshape((-1,1))
            test.loc[(test['Country/Region']==country)&(test['Province/State']==state),'Fatalities'] = model.predict(predict_x)

training model for country ==>US


In [21]:
df_train = train.drop(columns=['Id'])
df_test = test.drop(columns=['ForecastId'])
df_train['type'] = 'history'
df_test['type'] = 'forecast'
df_all = pd.concat([df_train, df_test])

grouped = df_all.groupby(['Date',  'type'])['Date', 'ConfirmedCases'].sum().reset_index()

fig = px.line(grouped, x="Date", y="ConfirmedCases", color = 'type', title="Taiwan Confirmed Cases Over Time")
# fig.update_traces(mode='markers+lines')


/Users/kdeng/opt/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [15]:
fig.show()

In [7]:
init_notebook_mode(connected=True)

In [18]:
test.tail()

,ForecastId,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
11691,11692,Wyoming,US,42.756,-107.3025,2020-04-19,20.439878,0.0
11692,11693,Wyoming,US,42.756,-107.3025,2020-04-20,21.086090,0.0
11693,11694,Wyoming,US,42.756,-107.3025,2020-04-21,21.741952,0.0
11694,11695,Wyoming,US,42.756,-107.3025,2020-04-22,22.407463,0.0
11695,11696,Wyoming,US,42.756,-107.3025,2020-04-23,23.082624,0.0


In [6]:
from plotly.offline import init_notebook_mode, iplot

In [20]:
iplot(fig)

In [14]:
init_notebook_mode(connected=True)